In [1]:
import pandas as pd
import numpy as np
import main_classificaiton as mc
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import os
import random


In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. w/o data representation & RNN model 
config1 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'model': 'RNN', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

# Case 2. w/o data representation &LSTM model 
config2 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'model': 'LSTM', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

# Case 3. w/o data representation & GRU model 
config3 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'model': 'GRU', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'num_layers' : 2, # recurrnet layers의 수, Default : 1
            'hidden_size' : 64, # hidden state의 벡터차원 수
            'attention' : False, # True일 경우 attention layer를 추가
            'dropout' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'bidirectional' : True, # 모델의 양방향성 여부
            'batch_size' : 64, #batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

# Case 4. w/o data representation & CNN_1D model 
config4 = {
        'with_representation': False, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'model': 'CNN_1D', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC_layer'} 중 택 1

        'parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'output_channels' : 64, # convolution channel size of output
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'kernel_size' : 3, # convolutional filter size
            'stride' : 1, # stride of the convolution. Default = 1 
            'padding' : 0, # padding added to both sides of the input. Default = 0
            'batch_size' : 64, # batch size
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

# Case 5. w data representation & fully-connected layers 
# 특징 벡터의 사이즈 = 20 이라고 가정
config5 = {
        'with_representation': True, # classification에 사용되는 representation이 있을 경우 True, 아닐 경우 False
        'model': 'FC', # classification에에 활용할 알고리즘 정의, {'RNN', 'LSTM', 'GRU', 'CNN_1D', 'FC'} 중 택 1

        'parameter': {
            'window_size' : 50, # input time series data를 windowing 하여 자르는 길이(size)
            'input_representation' : 0, # 예를 들면 (80, 20) 차원의 벡터 (80은 window_length에 따른 관측치 수, 20은 representation 특징벡터 차원 수)를 넣어야 함. 지금은 loader부분에서 random값들어가 있음
            'drop_out' : 0.2, # If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
            'batch_size' : 64, # batch size
            'bias': True, # bias [True, False]
            'device': 'cuda', # 학습 환경, ["cuda", "cpu"] 중 선택
            'num_epochs' : 100 # 학습 시 사용할 epoch 수
            }
}

In [4]:
data_dir = './data'

train_x = pickle.load(open(os.path.join(data_dir, 'x_train.pkl'), 'rb'))
train_y = pickle.load(open(os.path.join(data_dir, 'state_train.pkl'), 'rb'))
test_x =  pickle.load(open(os.path.join(data_dir, 'x_test.pkl'), 'rb'))
test_y = pickle.load(open(os.path.join(data_dir, 'state_test.pkl'), 'rb'))

train_data = {'x' : train_x, 'y' : train_y}
test_data = {'x' : test_x, 'y' : test_y}

In [5]:
# Case 1. w/o data representation & RNN
config = config1
data_classification = mc.Classification(config, train_data, test_data)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7531 Acc: 0.1375
val Loss: 1.6371 Acc: 0.2400

Epoch 2/100
----------
train Loss: 1.6573 Acc: 0.2000
val Loss: 1.5537 Acc: 0.4400

Epoch 3/100
----------
train Loss: 1.5801 Acc: 0.4125
val Loss: 1.4827 Acc: 0.5200

Epoch 4/100
----------
train Loss: 1.5103 Acc: 0.4250
val Loss: 1.4258 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.4578 Acc: 0.4250
val Loss: 1.3827 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.4128 Acc: 0.4125
val Loss: 1.3508 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.3764 Acc: 0.4250
val Loss: 1.3260 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.3456 Acc: 0.4250
val Loss: 1.3052 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.3193 Acc: 0.4250
val Loss: 1.2880 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.2957 Acc: 0.4250
val Loss: 1.2733 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.2743 Acc: 0.4625
val Loss: 1.2591 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.2574 Acc: 0.4875
val Loss: 1.2455 Ac

val Loss: 0.9801 Acc: 0.6000

Epoch 100/100
----------
train Loss: 0.6856 Acc: 0.7500
val Loss: 0.9596 Acc: 0.6400

Training complete in 0m 3s
Best val Acc: 0.760000

accuracy of test set : 0.6888888888888889


In [6]:
# Case 2. w/o data representation & LSTM
config = config2
data_classification = mc.Classification(config, train_data, test_data)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7991 Acc: 0.1000
val Loss: 1.7827 Acc: 0.1200

Epoch 2/100
----------
train Loss: 1.7749 Acc: 0.2375
val Loss: 1.7573 Acc: 0.5200

Epoch 3/100
----------
train Loss: 1.7513 Acc: 0.4625
val Loss: 1.7338 Acc: 0.5600

Epoch 4/100
----------
train Loss: 1.7300 Acc: 0.5000
val Loss: 1.7112 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.7095 Acc: 0.5000
val Loss: 1.6900 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.6910 Acc: 0.5000
val Loss: 1.6703 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.6723 Acc: 0.4875
val Loss: 1.6519 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.6549 Acc: 0.4750
val Loss: 1.6340 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.6382 Acc: 0.4625
val Loss: 1.6162 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.6215 Acc: 0.4625
val Loss: 1.5979 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.6049 Acc: 0.4750
val Loss: 1.5791 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.5876 Acc: 0.4750
val Loss: 1.5600 Ac

val Loss: 0.5857 Acc: 0.8800

Epoch 98/100
----------
train Loss: 0.3006 Acc: 0.9875
val Loss: 0.5807 Acc: 0.8800

Epoch 99/100
----------
train Loss: 0.2944 Acc: 0.9875
val Loss: 0.5811 Acc: 0.8800

Epoch 100/100
----------
train Loss: 0.2857 Acc: 1.0000
val Loss: 0.5845 Acc: 0.8800

Training complete in 0m 3s
Best val Acc: 0.880000

accuracy of test set : 0.7555555555555555


In [7]:
# Case 3. w/o data representation & GRU
config = config3
data_classification = mc.Classification(config, train_data, test_data)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.7450 Acc: 0.2250
val Loss: 1.7232 Acc: 0.1200

Epoch 2/100
----------
train Loss: 1.6805 Acc: 0.2875
val Loss: 1.6594 Acc: 0.1200

Epoch 3/100
----------
train Loss: 1.6253 Acc: 0.2875
val Loss: 1.6013 Acc: 0.1200

Epoch 4/100
----------
train Loss: 1.5753 Acc: 0.3500
val Loss: 1.5481 Acc: 0.5600

Epoch 5/100
----------
train Loss: 1.5318 Acc: 0.4875
val Loss: 1.5011 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.4900 Acc: 0.4375
val Loss: 1.4598 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.4565 Acc: 0.4250
val Loss: 1.4234 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.4243 Acc: 0.4250
val Loss: 1.3921 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.3952 Acc: 0.4250
val Loss: 1.3648 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.3708 Acc: 0.4250
val Loss: 1.3417 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.3483 Acc: 0.4250
val Loss: 1.3227 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.3286 Acc: 0.4250
val Loss: 1.3074 Ac

val Loss: 0.6492 Acc: 0.8800

Epoch 98/100
----------
train Loss: 0.5569 Acc: 0.8250
val Loss: 0.6425 Acc: 0.8800

Epoch 99/100
----------
train Loss: 0.5515 Acc: 0.8125
val Loss: 0.6367 Acc: 0.8800

Epoch 100/100
----------
train Loss: 0.5459 Acc: 0.8125
val Loss: 0.6319 Acc: 0.8800

Training complete in 0m 3s
Best val Acc: 0.880000

accuracy of test set : 0.7555555555555555


In [9]:
# Case 4. w/o data representation & CNN_1D
config = config4
data_classification = mc.Classification(config, train_data, test_data)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.8168 Acc: 0.1000
val Loss: 1.7620 Acc: 0.1600

Epoch 2/100
----------
train Loss: 1.7459 Acc: 0.3000
val Loss: 1.6989 Acc: 0.6400

Epoch 3/100
----------
train Loss: 1.6922 Acc: 0.5500
val Loss: 1.6475 Acc: 0.5200

Epoch 4/100
----------
train Loss: 1.6510 Acc: 0.4500
val Loss: 1.5969 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.5970 Acc: 0.3750
val Loss: 1.5480 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.5365 Acc: 0.4375
val Loss: 1.4957 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.4955 Acc: 0.4125
val Loss: 1.4400 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.4359 Acc: 0.4250
val Loss: 1.3881 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.3855 Acc: 0.4125
val Loss: 1.3460 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.3374 Acc: 0.4500
val Loss: 1.3125 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.3169 Acc: 0.4250
val Loss: 1.2837 Acc: 0.5200

Epoch 12/100
----------
train Loss: 1.2935 Acc: 0.4250
val Loss: 1.2600 Ac

val Loss: 0.7661 Acc: 0.8800

Epoch 99/100
----------
train Loss: 0.2864 Acc: 0.9125
val Loss: 0.7632 Acc: 0.8800

Epoch 100/100
----------
train Loss: 0.3177 Acc: 0.9125
val Loss: 0.7606 Acc: 0.8800

Training complete in 0m 1s
Best val Acc: 0.880000

accuracy of test set : 0.8222222222222223


In [10]:
# Case 5. w/ data representation & fully-connected layers
config = config5
data_classification = mc.Classification(config, train_data, test_data)
test_accuracy = data_classification.getResult()

print('\naccuracy of test set : {}'.format(test_accuracy))

Epoch 1/100
----------
train Loss: 1.9038 Acc: 0.0875
val Loss: 1.8807 Acc: 0.0800

Epoch 2/100
----------
train Loss: 1.8821 Acc: 0.1000
val Loss: 1.8803 Acc: 0.0800

Epoch 3/100
----------
train Loss: 1.8841 Acc: 0.0875
val Loss: 1.8799 Acc: 0.0800

Epoch 4/100
----------
train Loss: 1.8848 Acc: 0.0625
val Loss: 1.8795 Acc: 0.0800

Epoch 5/100
----------
train Loss: 1.8829 Acc: 0.1000
val Loss: 1.8790 Acc: 0.0800

Epoch 6/100
----------
train Loss: 1.8907 Acc: 0.0500
val Loss: 1.8786 Acc: 0.0800

Epoch 7/100
----------
train Loss: 1.8637 Acc: 0.1125
val Loss: 1.8782 Acc: 0.0800

Epoch 8/100
----------
train Loss: 1.8715 Acc: 0.1125
val Loss: 1.8778 Acc: 0.0800

Epoch 9/100
----------
train Loss: 1.8866 Acc: 0.0875
val Loss: 1.8773 Acc: 0.0800

Epoch 10/100
----------
train Loss: 1.8805 Acc: 0.0875
val Loss: 1.8769 Acc: 0.0800

Epoch 11/100
----------
train Loss: 1.8663 Acc: 0.0875
val Loss: 1.8765 Acc: 0.0800

Epoch 12/100
----------
train Loss: 1.8837 Acc: 0.1125
val Loss: 1.8761 Ac